# Fine-grained Clustering of Documents

## Import Dataset

In [43]:
import numpy as np
import pandas as pd
#import seaborn as sns; sns.set()

excel_df = pd.read_excel('corpus.xlsx', sheet_name='etalon export_file')

# print whole sheet data
excel_df.head()

,ID,Headline,Body
0,32652,В СГТУ пройдет конференция по современным биои...,"<div id=""content-content"">\n<div id=""node-2692..."
1,32651,Продолжается набор слушателей по программам ДПО,"<p style=""text-align: justify;""><img style=""fl..."
2,32650,Студент АМФ - серебряный призер Чемпионата Рос...,"<p style=""text-align: justify;""><img style=""fl..."
3,32643,Сегодня - День Государственного флага РФ,"<p style=""text-align: justify;""><img style=""fl..."
4,58784,Кафедра рекламы и компьютерного дизайна провод...,"\r\n<div style=""text-align: justify;"">Вниманию..."


## Preprocessing XLSX file 

#### Select a column Body

In [44]:
# find out how many rows are in the sheet
excel_df.shape

(2000, 3)

In [45]:
i = range(0, 2000)
body = excel_df.loc[i,['Body']]
body

,Body
0,"<div id=""content-content"">\n<div id=""node-2692..."
1,"<p style=""text-align: justify;""><img style=""fl..."
2,"<p style=""text-align: justify;""><img style=""fl..."
3,"<p style=""text-align: justify;""><img style=""fl..."
4,"\r\n<div style=""text-align: justify;"">Вниманию..."
...,...
1995,"<p style=""text-align: justify;""><img style=""fl..."
1996,"<p style=""text-align: justify;""><img style=""fl..."
1997,"<p style=""text-align: justify;""><img style=""fl..."
1998,"<p style=""text-align: justify;""><img style=""fl..."


#### Delete HTML tags and special chars

In [46]:
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

# Набор специальных символов, которые будут удалены из текста
spec_chars = string.punctuation + '\n\xa0«»\t—…'  


In [47]:
spec_charss = string.punctuation
remove_words = ['abc', 'div', 'ООО', '\n\xa0«»\t—…']


pat = r'\b(?:{})\b'.format('|'.join(remove_words))

pat

'\\b(?:abc|div|ООО|\n\xa0«»\t—…)\\b'

In [48]:
excel_df['new'] = excel_df['Body'].str.replace(pat, '')

excel_df

,ID,Headline,Body,new
0,32652,В СГТУ пройдет конференция по современным биои...,"<div id=""content-content"">\n<div id=""node-2692...","< id=""content-content"">\n< id=""node-26925"" cla..."
1,32651,Продолжается набор слушателей по программам ДПО,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
2,32650,Студент АМФ - серебряный призер Чемпионата Рос...,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
3,32643,Сегодня - День Государственного флага РФ,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
4,58784,Кафедра рекламы и компьютерного дизайна провод...,"\r\n<div style=""text-align: justify;"">Вниманию...","\r\n< style=""text-align: justify;"">Вниманию ст..."
...,...,...,...,...
1995,18527,Профессор СГТУ - победитель конкурса РФФИ,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
1996,18528,В санатории ?Чайка? начинается вторая смена,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
1997,18529,Юбилейный выпуск переводчиков получил дипломы,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."
1998,18530,В Волгограде прошла конференция в рамках проек...,"<p style=""text-align: justify;""><img style=""fl...","<p style=""text-align: justify;""><img style=""fl..."


In [53]:
body.str.lower()

excel_df

AttributeError: 'DataFrame' object has no attribute 'str'

#### Lemmatize the text

In [49]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

# Набор специальных символов, которые будут удалены из текста
spec_chars = string.punctuation + '\n\xa0«»\t—…'  

# text_tokens представляет собой список слов (токенов)
text_tokens = word_tokenize(text)   
print(text_tokens, type(text_tokens), len(text_tokens))

# Для применения инструментов частотного анализа библиотеки NLTK необходимо список токенов преобразовать к классу Text
text = nltk.Text(text_tokens)
print(type(text))

# Получим список стоп-слов для русского языка
russian_stopwords = stopwords.words("russian")  
russian_stopwords.extend(['это', 'и', 'нем'])

text_tokens = [token.strip() for token in text_tokens if token not in russian_stopwords]
print(text_tokens, type(text_tokens), len(text_tokens))

bag = list()

for text_token in text_tokens:
    p = morph.parse(text_token)[0]
    bag.append(p.normal_form)
print(bag)

NameError: name 'text' is not defined

## K-means

In [50]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# генерируем синтетические двумерные данные
X, y = make_blobs(random_state=1)

# строим модель кластеризации
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

KMeans(n_clusters=3)

## KD-trees

## LSH